<a href="https://colab.research.google.com/github/fact-h/Graduation-project/blob/main/LightGBM_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基于机器学习的城市洪涝快速模拟研究

- 目标：根据降雨和潮位的序列信息预测某点的最大水深
- 使用的机器学习算法：[LightGBM](https://lightgbm.readthedocs.io/en/latest/)
- 模型输入特征：10个降雨和潮位特征
- 模型输出变量：最大水深



## 导入相关的模块

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt

%matplotlib inline

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

## 数据预处理

### 加载数据集
先上传两个CSV数据文件：
- `E:\毕业设计\数据\模型训练数据\X.csv`
- `E:\毕业设计\数据\模型训练数据\y.csv`

In [2]:
df_X_raw = pd.read_csv('/content/X.csv')
df_y = pd.read_csv('/content/y.csv')

## 提取特征
10个特征：
- 6个降雨相关的特征：**累计降雨量 降雨重现期 降雨峰值 最大2h降雨量 最大3h降雨量 峰值前累计降雨量**
- 4个潮位相关的特征：**最大潮位 潮位重现期 平均潮位 最大5h平均潮位**

### 创建降雨的特征DataFrame: `rain_feature_df`和创建潮位的有关特征DataFrame: `tide_feature_df`

In [3]:
# 新建一个降雨DataFrame
rain_feature_df = pd.DataFrame()

# 添加累计降雨量
rain_feature_df['CumRainfall'] = df_X_raw.iloc[:,2:9].sum()

# 添加降雨重现期
rain_feature_df['RainRP'] = [5,10,20,35,50,75,100]

# 添加降雨峰值
rain_feature_df['RainfallPeak'] = df_X_raw.iloc[:,2:9].max()

# 添加最大2h降雨量
rain_feature_df['MaxRainfall2h'] = np.add(df_X_raw.iloc[0:-2,2:9], df_X_raw.iloc[1:-1,2:9]).max()

# 添加最大3h降雨量
rain_feature_df['MaxRainfall3h'] = np.add(np.add(df_X_raw.iloc[0:-3,2:9],df_X_raw.iloc[1:-2,2:9]),df_X_raw.iloc[2:-1,2:9]).max()

# 添加峰值前累计降雨量
peak_index = df_X_raw[df_X_raw.iloc[:,2]==rain_feature_df['RainfallPeak'][0]].index.tolist()[0]
rain_feature_df['CumRainfallBeforePeak'] = df_X_raw.iloc[0:peak_index,2:9].sum()



# 创建一个从潮位中提取的特征DataFrame
tide_feature_df = pd.DataFrame()

# 添加最大潮位
tide_feature_df['MaxTide'] = df_X_raw.iloc[:,9:].max()

# 添加潮位重现期
tide_feature_df['TideRP'] = [5,10,20,35,50,75,100]

# 添加平均潮位
tide_feature_df['MeanTide'] = df_X_raw.iloc[:,9:].mean()

# 添加最大5h平均潮位
tide_feature_df['MaxTide5h'] = np.add(
    np.add(np.add(
        np.add(df_X_raw.iloc[0:-5,9:],df_X_raw.iloc[1:-4,9:]),
        df_X_raw.iloc[2:-3,9:]),df_X_raw.iloc[3:-2,9:]),
        df_X_raw.iloc[4:-1,9:]).max()/5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)'

## 混合降雨和潮位数据

### 重置索引，将`rainx`或`tidex`换为数字

In [4]:
# 重置索引，将索引换为数字形式，方便后面数据组合
rain_feature_df = rain_feature_df.reset_index(drop=True) # 重置索引后将原索引所在的列删除
tide_feature_df = tide_feature_df.reset_index(drop=True)

### 将降雨特征、潮位特征和最大水深组合在一起，形成49条数据样本
先将降雨的每一行复制7遍，再用`concat`方法将7组潮位数据首尾相连，即相当于整个复制7遍。

然后使用`join`方法连接降雨和潮位，每个重现期的降雨对应7个重现期的潮位。最后再将水深数据加上，得到总的数据集`df_data`。

In [5]:
# 将数据的每一行复制7遍
rain_repeat_df = pd.DataFrame(np.repeat(rain_feature_df.values,tide_feature_df.shape[0],axis=0)) 
rain_repeat_df.columns = rain_feature_df.columns
# 将所有数据复制7遍
tide_concat_df = pd.concat([tide_feature_df, tide_feature_df, tide_feature_df, tide_feature_df, tide_feature_df, tide_feature_df, tide_feature_df]).reset_index(drop=True) 

# 组合降雨和潮位特征数据
df_X = rain_repeat_df.join(tide_concat_df)
# 添加输出变量-水深
df_data = df_X.join(df_y['depth'])
df_data.head()

,CumRainfall,RainRP,RainfallPeak,MaxRainfall2h,MaxRainfall3h,CumRainfallBeforePeak,MaxTide,TideRP,MeanTide,MaxTide5h,depth
0,199.101882,5.0,56.394737,86.377019,99.455229,35.806351,2.898900,5,1.872049,2.579491,0.34
1,199.101882,5.0,56.394737,86.377019,99.455229,35.806351,3.159600,10,2.040403,2.811467,0.39
2,199.101882,5.0,56.394737,86.377019,99.455229,35.806351,3.400900,20,2.196230,3.026179,0.43
3,199.101882,5.0,56.394737,86.377019,99.455229,35.806351,3.585497,35,2.315439,3.190437,0.48
4,199.101882,5.0,56.394737,86.377019,99.455229,35.806351,3.701200,50,2.390157,3.293392,0.53


In [ ]:
# 归一化：z-score
df_data_mean = df_data.mean()
df_data_std = df_data.std()
df_data_norm = (df_data - df_data.mean()) / df_data.std()
df_data_norm

In [ ]:
# 加入正态分布进行数据增强
df_data_augmented = df_data_norm + 0.005 * df_data_norm * np.random.standard_normal(size=df_data_norm.shape)
df_data_augmented

In [ ]:
# 将添加过噪声的数据加到总数据集中，并打乱数据
df_all_data = pd.concat([df_data_norm,df_data_augmented])
df_all_data = df_all_data.reset_index(drop=True)
df_all_data = df_all_data.reindex(np.random.permutation(df_all_data.index))
df_all_data = df_all_data.reset_index(drop=True)
df_all_data

In [26]:
# 创建训练集、验证集和测试集
train_split = round(0.7 * df_all_data.shape[0])
val_split = round(0.2 * df_all_data.shape[0])
test_split = round(0.1 * df_all_data.shape[0])

y_test = df_all_data.depth[0:test_split]
y_val = df_all_data.depth[test_split:(val_split + test_split)]
y_train = df_all_data.depth[(val_split + test_split):]

X_test = df_all_data[0:test_split].drop(['depth'], axis=1)
X_val = df_all_data[test_split:(val_split + test_split)].drop(['depth'], axis=1)
X_train = df_all_data[(val_split + test_split):].drop(['depth'], axis=1)

# 进行训练

In [ ]:
evals_result = {} # 为绘图记录评估结果
gbm = lgb.LGBMRegressor(num_leaves=30,
                        learning_rate=0.01,
                        n_estimators=200)
gbm.fit(X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric=['l1','l2'],
        callbacks=[lgb.early_stopping(5), lgb.record_evaluation(evals_result)])

# 绘制训练过程中的指标变化

In [27]:
def render_metric(metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    plt.show()

if INTERACTIVE:
    # create widget to switch between metrics
    interact(render_metric, metric_name=['l1','l2'])
else:
    render_metric('l1')

interactive(children=(Dropdown(description='metric_name', options=('l1', 'l2'), value='l1'), Output()), _dom_c…

# 绘制特征重要性

In [16]:
def render_plot_importance(importance_type, max_features=10,
                           ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    plt.show()

if INTERACTIVE:
    # create widget for interactive feature importance plot
    interact(render_plot_importance,
             importance_type=['split', 'gain'],
             max_features=(1, X_train.shape[-1]),
             precision=(0, 10))
else:
    render_plot_importance(importance_type='split')

interactive(children=(Dropdown(description='importance_type', options=('split', 'gain'), value='split'), IntSl…

# 绘制直方图

In [20]:
def render_tree(tree_index, show_info, precision=3):
    show_info = None if 'None' in show_info else show_info
    return lgb.create_tree_digraph(gbm, tree_index=tree_index,
                                   show_info=show_info, precision=precision)

if INTERACTIVE:
    # create widget to switch between trees and control info in nodes
    interact(render_tree,
             tree_index=(0, gbm.n_estimators - 1),
             show_info=SelectMultiple(  # allow multiple values to be selected
                 options=['None',
                          'split_gain',
                          'internal_value',
                          'internal_count',
                          'internal_weight',
                          'leaf_count',
                          'leaf_weight',
                          'data_percentage'],
                 value=['None']),
             precision=(0, 10))
    tree = None
else:
    tree = render_tree(53, ['None'])
tree

interactive(children=(IntSlider(value=99, description='tree_index', max=199), SelectMultiple(description='show…

# 开始预测

In [28]:
# 预测
y_pred = gbm.predict(X_test,num_iteration=gbm.best_iteration_)

# 归一化后的评估
rmse_test = mean_squared_error(y_test,y_pred) ** 0.5 # mse加根号即是rmse
print(f'The RMSE of prediction is: {rmse_test}')

The RMSE of prediction is: 0.2811841873273574


In [29]:
# 原始数据评估
y_pred_raw = y_pred * df_data_std.depth + df_data_mean.depth
y_test_raw = y_test * df_data_std.depth + df_data_mean.depth

rmse_test_raw = mean_squared_error(y_test_raw,y_pred_raw) ** 0.5 # mse加根号即是rmse
print(f'The RMSE of raw prediction is: {rmse_test_raw}')

The RMSE of raw prediction is: 0.02697023978883221


In [30]:
depth = [list(y_test_raw),list(y_pred_raw)]

depth = np.transpose(depth)
cols = ['real','predict']
df = pd.DataFrame(data=depth,columns=cols)
df['relative_error'] = np.abs(df.predict - df.real)
df

,real,predict,relative_error
0,0.700000,0.702180,0.002180
1,0.660000,0.687104,0.027104
2,0.569983,0.588594,0.018611
3,0.579947,0.584969,0.005022
4,0.680000,0.635249,0.044751
5,0.690184,0.648357,0.041827
6,0.600000,0.597509,0.002491
7,0.490000,0.518485,0.028485
8,0.649960,0.612388,0.037572
9,0.590000,0.603474,0.013474


In [31]:
# 计算纳什效率系数
H_obs = y_test_raw
H_m = y_pred_raw
H_m_mean = H_obs.mean()

NSE = 1 - ((H_obs - H_m)**2).sum() / ((H_obs - H_m_mean)**2).sum()
print(f'The NSE of prediction is: {NSE}')

The NSE of prediction is: 0.8168656682543713


In [32]:
# 计算R2_score
R2 = r2_score(y_test_raw,y_pred_raw)
print(f'The R2 score of prediction is: {R2}')

The R2 score of prediction is: 0.8168656682543713
